<a href="https://colab.research.google.com/github/Vonewman/Deep-Learning-with-PyTorch/blob/main/feedforward_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
# Hyperparameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [6]:
class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.4237
Epoch [1/5], Step [200/600], Loss: 0.3756
Epoch [1/5], Step [300/600], Loss: 0.2071
Epoch [1/5], Step [400/600], Loss: 0.1589
Epoch [1/5], Step [500/600], Loss: 0.1322
Epoch [1/5], Step [600/600], Loss: 0.1060
Epoch [2/5], Step [100/600], Loss: 0.1127
Epoch [2/5], Step [200/600], Loss: 0.0518
Epoch [2/5], Step [300/600], Loss: 0.0921
Epoch [2/5], Step [400/600], Loss: 0.1014
Epoch [2/5], Step [500/600], Loss: 0.1160
Epoch [2/5], Step [600/600], Loss: 0.1769
Epoch [3/5], Step [100/600], Loss: 0.1023
Epoch [3/5], Step [200/600], Loss: 0.0478
Epoch [3/5], Step [300/600], Loss: 0.0757
Epoch [3/5], Step [400/600], Loss: 0.2094
Epoch [3/5], Step [500/600], Loss: 0.0637
Epoch [3/5], Step [600/600], Loss: 0.0595
Epoch [4/5], Step [100/600], Loss: 0.0754
Epoch [4/5], Step [200/600], Loss: 0.0432
Epoch [4/5], Step [300/600], Loss: 0.0367
Epoch [4/5], Step [400/600], Loss: 0.0428
Epoch [4/5], Step [500/600], Loss: 0.0514
Epoch [4/5], Step [600/600], Loss:

In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.85 %


In [11]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')